Урок 7. Многомерный статистический анализ. Линейная регрессия
Дана матрица объект-признак

X = [[ 1.22401313, 2.30868478, 3.03636353, 2.69287214],
[-0.18757272, 1.30337355, 5.12093014, 3.46363202],
[-0.81094525, 1.82463398, 5.79686488, 1.86159445],
[ 0.75129018, 2.67392052, 3.65529809, 1.66746094],
[ 0.00972362, 1.97367255, 2.50594319, 1.69755173],
[-0.62972637, 0.77750764, 2.84124027, 4.54410559],
[ 2.29536229, 1.81206697, 1.95026215, 1.51874636],
[ 0.0920418 , 2.26971361, 7.47708735, 2.61081203],
[ 2.39252799, 3.17563985, 3.61420599, 5.10773362],
[ 0.54983815, 2.87988651, 1.65752765, 1.59635987]]
и значения целевой переменной

y = [ 9.26193358, 9.700363 , 8.67214805, 8.74796974, 6.18689108,
7.53312713, 7.57643777, 12.44965478, 14.29010746, 6.68361218]
1. Подберите два признака (из четырёх) так, чтобы уровень линейной зависимости целевой переменной от значений этих признаков был максимальным. Другими словами, модель линейной регрессии на этих признаках должна давать наилучший результат.
2. Является ли значимым получившееся уравнение регрессии?

In [41]:
import numpy as np
import pandas as pd
from scipy import stats
from itertools import combinations

In [42]:
x = np.array([[ 1.22401313, 2.30868478, 3.03636353, 2.69287214], [-0.18757272, 1.30337355, 5.12093014, 3.46363202], [-0.81094525, 1.82463398, 5.79686488, 1.86159445], [ 0.75129018, 2.67392052, 3.65529809, 1.66746094], [ 0.00972362, 1.97367255, 2.50594319, 1.69755173], [-0.62972637, 0.77750764, 2.84124027, 4.54410559], [ 2.29536229, 1.81206697, 1.95026215, 1.51874636], [ 0.0920418 , 2.26971361, 7.47708735, 2.61081203], [ 2.39252799, 3.17563985, 3.61420599, 5.10773362], [ 0.54983815, 2.87988651, 1.65752765, 1.59635987]]).reshape(-1, 1)
x

array([[ 1.22401313],
       [ 2.30868478],
       [ 3.03636353],
       [ 2.69287214],
       [-0.18757272],
       [ 1.30337355],
       [ 5.12093014],
       [ 3.46363202],
       [-0.81094525],
       [ 1.82463398],
       [ 5.79686488],
       [ 1.86159445],
       [ 0.75129018],
       [ 2.67392052],
       [ 3.65529809],
       [ 1.66746094],
       [ 0.00972362],
       [ 1.97367255],
       [ 2.50594319],
       [ 1.69755173],
       [-0.62972637],
       [ 0.77750764],
       [ 2.84124027],
       [ 4.54410559],
       [ 2.29536229],
       [ 1.81206697],
       [ 1.95026215],
       [ 1.51874636],
       [ 0.0920418 ],
       [ 2.26971361],
       [ 7.47708735],
       [ 2.61081203],
       [ 2.39252799],
       [ 3.17563985],
       [ 3.61420599],
       [ 5.10773362],
       [ 0.54983815],
       [ 2.87988651],
       [ 1.65752765],
       [ 1.59635987]])

In [43]:
y = np.array([9.26193358, 9.700363 , 8.67214805, 8.74796974, 6.18689108, 7.53312713, 7.57643777, 12.44965478, 14.29010746, 6.68361218])
y

array([ 9.26193358,  9.700363  ,  8.67214805,  8.74796974,  6.18689108,
        7.53312713,  7.57643777, 12.44965478, 14.29010746,  6.68361218])

Транспонируем матрицу

In [44]:
xt = x.transpose()
print(xt)

[[ 1.22401313  2.30868478  3.03636353  2.69287214 -0.18757272  1.30337355
   5.12093014  3.46363202 -0.81094525  1.82463398  5.79686488  1.86159445
   0.75129018  2.67392052  3.65529809  1.66746094  0.00972362  1.97367255
   2.50594319  1.69755173 -0.62972637  0.77750764  2.84124027  4.54410559
   2.29536229  1.81206697  1.95026215  1.51874636  0.0920418   2.26971361
   7.47708735  2.61081203  2.39252799  3.17563985  3.61420599  5.10773362
   0.54983815  2.87988651  1.65752765  1.59635987]]


In [45]:
def sum_of_squares(samples):
    return ((samples - samples.mean()) ** 2).sum()

In [48]:
df = pd.DataFrame({'X1':[ 1.22401313, -0.18757272, -0.81094525,  0.75129018,  0.00972362,
        -0.62972637,  2.29536229,  0.0920418 ,  2.39252799,  0.54983815], 'X2':[ 2.30868478,  1.30337355,  1.82463398,  2.67392052,  1.97367255,
         0.77750764,  1.81206697,  2.26971361,  3.17563985,  2.87988651], 'X3':[ 3.03636353,  5.12093014,  5.79686488,  3.65529809,  2.50594319,
         2.84124027,  1.95026215,  7.47708735,  3.61420599,  1.65752765], 'X4':[ 2.69287214,  3.46363202,  1.86159445,  1.66746094,  1.69755173,
         4.54410559,  1.51874636,  2.61081203,  5.10773362,  1.59635987]}, columns = ['X1','X2', 'X3', 'X4'])
df

,X1,X2,X3,X4
0,1.224013,2.308685,3.036364,2.692872
1,-0.187573,1.303374,5.120930,3.463632
2,-0.810945,1.824634,5.796865,1.861594
3,0.751290,2.673921,3.655298,1.667461
4,0.009724,1.973673,2.505943,1.697552
5,-0.629726,0.777508,2.841240,4.544106
6,2.295362,1.812067,1.950262,1.518746
7,0.092042,2.269714,7.477087,2.610812
8,2.392528,3.175640,3.614206,5.107734
9,0.549838,2.879887,1.657528,1.596360


In [63]:
#list(combinations(range(6), r=2))

In [64]:
list(zip(x, y))[:10]

[(array([1.22401313]), 9.26193358),
 (array([2.30868478]), 9.700363),
 (array([3.03636353]), 8.67214805),
 (array([2.69287214]), 8.74796974),
 (array([-0.18757272]), 6.18689108),
 (array([1.30337355]), 7.53312713),
 (array([5.12093014]), 7.57643777),
 (array([3.46363202]), 12.44965478),
 (array([-0.81094525]), 14.29010746),
 (array([1.82463398]), 6.68361218)]

In [69]:
P = 0
Q = 0
i = 0

for pair1, pair2 in combinations(zip(x, y), r=2):
    if i < 10:
        print(x, y)
        i += 1
    else:
        Q += 1
        
P, Q

[[ 1.22401313]
 [ 2.30868478]
 [ 3.03636353]
 [ 2.69287214]
 [-0.18757272]
 [ 1.30337355]
 [ 5.12093014]
 [ 3.46363202]
 [-0.81094525]
 [ 1.82463398]
 [ 5.79686488]
 [ 1.86159445]
 [ 0.75129018]
 [ 2.67392052]
 [ 3.65529809]
 [ 1.66746094]
 [ 0.00972362]
 [ 1.97367255]
 [ 2.50594319]
 [ 1.69755173]
 [-0.62972637]
 [ 0.77750764]
 [ 2.84124027]
 [ 4.54410559]
 [ 2.29536229]
 [ 1.81206697]
 [ 1.95026215]
 [ 1.51874636]
 [ 0.0920418 ]
 [ 2.26971361]
 [ 7.47708735]
 [ 2.61081203]
 [ 2.39252799]
 [ 3.17563985]
 [ 3.61420599]
 [ 5.10773362]
 [ 0.54983815]
 [ 2.87988651]
 [ 1.65752765]
 [ 1.59635987]] [ 9.26193358  9.700363    8.67214805  8.74796974  6.18689108  7.53312713
  7.57643777 12.44965478 14.29010746  6.68361218]
[[ 1.22401313]
 [ 2.30868478]
 [ 3.03636353]
 [ 2.69287214]
 [-0.18757272]
 [ 1.30337355]
 [ 5.12093014]
 [ 3.46363202]
 [-0.81094525]
 [ 1.82463398]
 [ 5.79686488]
 [ 1.86159445]
 [ 0.75129018]
 [ 2.67392052]
 [ 3.65529809]
 [ 1.66746094]
 [ 0.00972362]
 [ 1.97367255]
 [ 2.5

(0, 35)

что-то видимо пошло не так...

это все, что я смогла на гране своих умственных способностей... и даже за гранью..